# Notebook 2: Pruebas del Buscador y Agente Completo

Este notebook prueba las herramientas de búsqueda (FAQ y Reglamentos) y el agente completo con el selector de LLM.

In [ ]:
# Importar las funciones
import sys
import os

# Añadimos la carpeta 'src' al path para que Python la encuentre
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

from tools.buscador import buscar_en_faq, buscador_de_reglamentos


In [3]:
print("--- Probando Buscador de FAQ ---")

pregunta_faq_1 = "¿dónde encuentro el correo de soporte?"
print(f"Pregunta: {pregunta_faq_1}")
print(f"Respuesta: {buscar_en_faq(pregunta_faq_1)}\n")

pregunta_faq_2 = "¿cómo hago para cancelar una materia?"
print(f"Pregunta: {pregunta_faq_2}")
print(f"Respuesta: {buscar_en_faq(pregunta_faq_2)}\n")

pregunta_faq_3 = "¿a qué hora abre la biblioteca?"
print(f"Pregunta: {pregunta_faq_3}")
print(f"Respuesta: {buscar_en_faq(pregunta_faq_3)}\n")


--- Probando Buscador de FAQ ---
Pregunta: ¿dónde encuentro el correo de soporte?
Respuesta: El correo de soporte técnico es soporte.ti@une.edu.py.

Pregunta: ¿cómo hago para cancelar una materia?
Respuesta: Para cancelar una materia, debes completar el formulario de solicitud en la secretaría académica antes de la fecha límite establecida en el calendario académico.

Pregunta: ¿a qué hora abre la biblioteca?
Respuesta: La biblioteca está abierta de lunes a viernes de 8:00 a 21:00 hs.



## Prueba del Agente Completo con Buscadores

Ahora vamos a probar el agente completo que puede usar todas las herramientas disponibles.

In [ ]:
# Importar el código del agente
from dotenv import load_dotenv
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.tools import tool, Tool
from langchain.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

# Reusar las definiciones de herramientas
@tool
def calculadora_academica(consulta: str) -> str:
    """Útil para cuando el usuario pide calcular el promedio de varias notas."""
    from tools.calculadora import calcular_promedio_de_notas
    return calcular_promedio_de_notas(consulta)

@tool
def buscador_faq_tool(consulta: str) -> str:
    """Útil para responder preguntas frecuentes sobre temas generales como horarios o correos."""
    return buscar_en_faq(consulta)

def crear_agente():
    herramienta_buscador_reglamentos = Tool(
        name="buscador_reglamentos",
        func=buscador_de_reglamentos.buscar,
        description="Útil para responder preguntas específicas sobre el reglamento académico, como reglas de asistencia, calificaciones, o condiciones de examen."
    )
    tools = [calculadora_academica, buscador_faq_tool, herramienta_buscador_reglamentos]

    # Selector de LLM
    LLM_PROVIDER = "GOOGLE"  # Cambiar a "HUGGINGFACE" si prefieres
    print(f"--- Usando el proveedor de LLM: {LLM_PROVIDER} ---")

    if LLM_PROVIDER == "GOOGLE":
        llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.1)
    elif LLM_PROVIDER == "HUGGINGFACE":
        llm_endpoint = HuggingFaceEndpoint(
            repo_id="mistralai/Mistral-7B-Instruct-v0.2",
            task="text-generation",
            max_new_tokens=256,
            temperature=0.1,
            stop_sequences=["\nObservation:", "\nThought:"]
        )
        llm = ChatHuggingFace(llm=llm_endpoint)
    else:
        raise ValueError(f"Proveedor '{LLM_PROVIDER}' no reconocido.")

    template = """
    Eres un asistente que responde preguntas usando herramientas. Sigue estas reglas ESTRICTAMENTE.
    Para ayudarte, tienes acceso a las siguientes herramientas:
    {tools}
    REGLAS DE FORMATO:
    1.  Tu primer paso DEBE seguir este formato:
        Question: La pregunta que debes responder.
        Thought: Tu análisis de la pregunta y qué herramienta usar.
        Action: La herramienta a usar, debe ser una de [{tool_names}].
        Action Input: La entrada para la herramienta.
    2.  Después de `Action Input`, DETENTE. No escribas nada más. El sistema te dará la `Observation`.
    3.  Después de recibir una `Observation`, si ya tienes la respuesta, DEBES usar este formato:
        Thought: Ya tengo la respuesta final.
        Final Answer: Tu respuesta final a la pregunta del usuario.
    4.  REGLA CRÍTICA: Si la `Observation` indica que no se encontró la información, tu respuesta final DEBE ser que no pudiste encontrar la información. No inventes respuestas.
    --- EJEMPLO DE USO ---
    Question: ¿cuál es el promedio de 5 y 4?
    Thought: El usuario quiere calcular un promedio. Debo usar la herramienta 'calculadora_academica'.
    Action: calculadora_academica
    Action Input: 5, 4
    --- FIN DEL EJEMPLO ---
    Ahora, empieza.
    Question: {input}
    {agent_scratchpad}
    """

    prompt = PromptTemplate.from_template(template)
    agent = create_react_agent(llm, tools, prompt)
    
    return AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=True,
        handle_parsing_errors=True,
        max_iterations=5,
    )

print("✓ Agente configurado correctamente")

In [ ]:
# Crear el agente y probar con preguntas sobre FAQ
agente = crear_agente()

# Prueba 1: Pregunta sobre FAQ
pregunta1 = "¿A qué correo puedo escribir si tengo dudas?"
print(f"\n{'='*60}")
print(f"Pregunta 1: {pregunta1}")
print(f"{'='*60}\n")
respuesta1 = agente.invoke({"input": pregunta1})
print(f"\n{'='*60}")
print(f"Respuesta: {respuesta1['output']}")
print(f"{'='*60}\n")

In [ ]:
# Prueba 2: Pregunta sobre Reglamentos
pregunta2 = "¿Cuál es la nota mínima para aprobar una materia?"
print(f"\n{'='*60}")
print(f"Pregunta 2: {pregunta2}")
print(f"{'='*60}\n")
respuesta2 = agente.invoke({"input": pregunta2})
print(f"\n{'='*60}")
print(f"Respuesta: {respuesta2['output']}")
print(f"{'='*60}\n")

In [ ]:
# Prueba 3: Pregunta que requiere la calculadora
pregunta3 = "Si saqué 7, 8 y 6 en mis tres parciales, ¿cuál es mi promedio?"
print(f"\n{'='*60}")
print(f"Pregunta 3: {pregunta3}")
print(f"{'='*60}\n")
respuesta3 = agente.invoke({"input": pregunta3})
print(f"\n{'='*60}")
print(f"Respuesta: {respuesta3['output']}")
print(f"{'='*60}\n")